# Linear regression

This notebook compares various linear regression implementations. The dataset used is the [Toulouse bike sharing dataset](https://river-ml.github.io/generated/river.datasets.fetch_bikes.html#river.datasets.fetch_bikes).

In [1]:
%load_ext watermark
%watermark --python --machine --packages river,keras,sklearn,tensorflow,torch,vowpalwabbit --datename

Using TensorFlow backend.


Fri Jul 10 2020 

CPython 3.7.7
IPython 7.16.1

river 0.6.1
keras 2.3.1
sklearn 0.23.1
tensorflow 2.0.0
torch 1.5.1
vowpalwabbit 8.8.1

compiler   : Clang 4.0.1 (tags/RELEASE_401/final)
system     : Darwin
release    : 19.5.0
machine    : x86_64
processor  : i386
CPU cores  : 12
interpreter: 64bit


In [2]:
from river import compose
from river import datasets
from river import feature_extraction
from river import linear_model
from river import metrics
from river import optim
from river import preprocessing
from river import stats
from keras import layers
from keras import models
from keras import optimizers
from sklearn import linear_model as sk_linear_model
import torch

%run utils.py
%run wrappers.py

In [ ]:
n_features = 6
lr = 0.005

class PyTorchNet(torch.nn.Module):
    
    def __init__(self, n_features):
        super().__init__()
        self.linear = torch.nn.Linear(n_features, 1)
        torch.nn.init.constant_(self.linear.weight, 0)
        torch.nn.init.constant_(self.linear.bias, 0)
        
    def forward(self, x):
        return self.linear(x)
    
torch_model = PyTorchNet(n_features=n_features)

# Keras
inputs = layers.Input(shape=(n_features,))
predictions = layers.Dense(1, kernel_initializer='zeros', bias_initializer='zeros')(inputs)
keras_model = models.Model(inputs=inputs, outputs=predictions)
keras_model.compile(optimizer=optimizers.SGD(lr=lr), loss='mean_squared_error')


def add_hour(x):
    x['hour'] = x['moment'].hour
    return x

results = benchmark(
    get_X_y=datasets.fetch_bikes,
    n=182470,
    get_pp=lambda: (
        compose.Select('clouds', 'humidity', 'pressure', 'temperature', 'wind') +
        (
            add_hour |
            feature_extraction.TargetAgg(by=['station', 'hour'], how=stats.Mean())
        ) |
        preprocessing.StandardScaler()
    ),
    models=[
         ('river', 'LinearRegression', linear_model.LinearRegression(
            optimizer=optim.SGD(lr),
            l2=0.,
            intercept_lr=lr
        )),
        ('scikit-learn', 'SGDRegressor', ScikitLearnRegressor(
            model=sk_linear_model.SGDRegressor(
                learning_rate='constant',
                eta0=lr,
                penalty='none'
            ),
        )),
        ('PyTorch (CPU)', 'Linear', PyTorchRegressor(
            network=torch_model,
            loss_fn=torch.nn.MSELoss(),
            optimizer=torch.optim.SGD(torch_model.parameters(), lr=lr)
        )),
        ('Keras on Tensorflow (CPU)', 'Dense', KerasRegressor(
            model=keras_model
        )),
        ('Vowpal Wabbit', '', VowpalWabbitRegressor(
            loss_function='squared',
            sgd=True,
            learning_rate=lr,
            adaptive=False,
            normalized=False,
            invariant=False,
            initial_weight=0
            l2=0.,
            l1=0.,
            power_t=0
        ))
    ],
    get_metric=metrics.MSE
)

In [15]:
results

,Library,Model,MSE,Fit time,Average fit time,Predict time,Average predict time
0,Vowpal Wabbit,,79.588114,"12s, 935ms, 475μs, 72ns","70μs, 891ns","26s, 721ms, 149μs, 367ns","146μs, 441ns"


In [7]:
results

,Library,Model,MSE,Fit time,Average fit time,Predict time,Average predict time
0,river,LinearRegression,23.035085,"3s, 338ms, 465μs, 619ns","18μs, 296ns","892ms, 571μs, 786ns","4μs, 892ns"
1,scikit-learn,SGDRegressor,25.295369,"55s, 687ms, 660μs, 259ns","305μs, 188ns","19s, 876ms, 42μs, 73ns","108μs, 928ns"
2,PyTorch (CPU),Linear,23.035086,"1m, 45s, 290ms, 48μs, 287ns","577μs, 27ns","34s, 224ms, 26μs, 443ns","187μs, 560ns"
3,Keras on Tensorflow (CPU),Dense,23.035086,"3m, 40s, 447ms, 280μs, 203ns","1ms, 208μs, 129ns","2m, 11s, 758ms, 82μs, 511ns","722μs, 81ns"
